In [1]:
import sqlite3
import pandas as pd

Я для удобства презентации сделал в питоне, но ты на это не морочся - просто анализируй запрос и комменты к нему.

Считаем таблицу из excel.

In [23]:
df = pd.read_excel(u'Тестовое задание_Азбука Вкуса_2.xlsx',sheetname='Sheet1')
df.head()

,week,date,client_id,receipt_id,item_id,volume,amount
0,4,23.01.15,1-2CM-1,1-1OI2VZF,ТОВ244730,84.0,1
1,4,23.01.15,1-2CM-1,1-1OI2VZF,ТОВ244732,64.0,1
2,4,23.01.15,1-2CM-1,1-1OI2VZF,ТОВ228962,278.0,2
3,4,23.01.15,1-2CM-1,1-1OI2VZF,ТОВ223367,78.0,1
4,4,23.01.15,1-2CM-1,1-1OI2VZF,ТОВ279930,56.8,1


Создадим базу данных.

In [3]:
import csv, sqlite3
conn = sqlite3.connect("database1.db")
cur = conn.cursor()

Загрузим в нее датафрейм (таблицу с данными) и назовем таблицу sales.

In [33]:
df.to_sql('sales',conn,index=False,if_exists='replace')

Протестируем работают ли запросы.

In [34]:
sql = "select * from sales"
cur.execute(sql)
pd.DataFrame(cur.fetchall()).head()

,0,1,2,3,4,5,6
0,4,23.01.15,1-2CM-1,1-1OI2VZF,ТОВ244730,84.0,1
1,4,23.01.15,1-2CM-1,1-1OI2VZF,ТОВ244732,64.0,1
2,4,23.01.15,1-2CM-1,1-1OI2VZF,ТОВ228962,278.0,2
3,4,23.01.15,1-2CM-1,1-1OI2VZF,ТОВ223367,78.0,1
4,4,23.01.15,1-2CM-1,1-1OI2VZF,ТОВ279930,56.8,1


Все работает, переходим к заданию.

## Написать sql-скрипт, который будет выводить кол-во чеков на 3-ей неделе, содержащих 1, 2, 3 и более 3-х позиций (товаров) в чеке.

In [35]:
df.columns

Index(['week', 'date', 'client_id', 'receipt_id', 'item_id', 'volume',
       'amount'],
      dtype='object')

Cначала создадим запрос, который считает количество позиций (товаров) в каждом чеке и отбирает те, где это количество равно определенному значению (в данном запросе 1). Этот запрос эквивалентен сводной табцице построенной на исходных данных, где в строках стоит id чека, в значениях количество по id товара и фильтр по неделе (=3) - обязательно попробуй сделать ее сам!!! Далее это сводная таблица фильтруется по получившимуся количеству товаров (в этом запросе оно = 1).

По порядку опишу каждую строку:

1. select - каждый запрос, который делает запрос к существующей таблице начинается c select. Может начинаться с например create table для создания таблицы или drop table для удаление теблицы. Загугли какие еще могут быть.
2. После селекта идет список столбцов, которые ты хочешь получить в результате запроса. В данном случае это:
    - t1.receipt_id - id чека
    - count(t1.item_id) - количество позичиций (товаров) в чеке. Вместо count может быть  sum, avg, min, max - это функция аггрегации.
3. from sales t1 - после списка столбцов идет название таблицы в котором храняться эти столбцы, в данном случае это sales. Обрати внимание, что после sales я написал t1 - это псевдоним (алиас) таблицы sales. Это нужно для того, чтобы каждый раз не писать sales, а в место этого писать короткий псевдоним t1. Этот псевдоним используется во всех частях запроса.
4. where t1.week = 3 - в блоке where идут условия, по которым мы фильтруем запрос. В данном случаее от нас хотят данные только третьей недели.
5. group by t1.receipt_id - в блоке group by указываются поля по которым мы группируем (эквивалентно полям, которые добавляются в строки сводной таблицы)
6. having count(t1.item_id) = 1 - блок having позволяет фильтровать запрос по аггрегированному показателю - в данном случае по количеству товаров на каждый чек. В экселе бы ты скопировал свод на другой лист и фильтранул по необходимому количеству. Ну или наложил бы числовой фильтр по количеству прямо в сводной таблице.

In [85]:
sql = '''
select

t1.receipt_id,
count(t1.item_id)

from sales t1

where t1.week = 3

group by t1.receipt_id

having count(t1.item_id) = 1
'''

Запустим запрос и посмотрим, что он дал.

In [89]:
cur.execute(sql)
pd.DataFrame(cur.fetchall())

,0,1
0,1-1J4FRFN,1
1,1-1J64C96,1
2,1-1J7TED2,1
3,1-1J7XND6,1
4,1-1J827W9,1
5,1-1J83Z7A,1
6,1-1J8DG23,1
7,1-1J8M7LU,1
8,1-1J97RAC,1
9,1-1J9PNAU,1


В задании необходимо найти количество чеков, в которых заданное число товаров. Для этого просто нужно посчитать количество строк в получившейся таблице. В sql для этого мы сделаем запрос к запросу, который был создан выше. Запрос будет очень простым:

select count(*) from

Далее в скобках идет запрос созданный в предыдыщем пункте.

Все это и есть финальный запрос.

In [90]:
sql2 = '''

select count(*) from

(select

t1.receipt_id,
count(t1.item_id)

from sales t1

where
t1.week = 3

group by
t1.receipt_id

having count(t1.item_id) = 1)
'''

Проверим, как он работет для количества предметов = 1 (having count(t1.item_id) = 1).

In [92]:
cur.execute(sql2)
pd.DataFrame(cur.fetchall())

,0
0,36


Получилось 36 предметов. Далее найдем кол-во чеков для кол-во предметов = 2, =3, >3. Просто будем менять условие having. Это и есть ответы на задание.

Для 2.

In [93]:
sql2 = '''

select count(*) from

(select

t1.receipt_id,
count(t1.item_id)

from sales t1

where
t1.week = 3

group by
t1.receipt_id

having count(t1.item_id) = 2)
'''
cur.execute(sql2)
pd.DataFrame(cur.fetchall())

,0
0,35


Для 3

In [94]:
sql2 = '''

select count(*) from

(select

t1.receipt_id,
count(t1.item_id)

from sales t1

where
t1.week = 3

group by
t1.receipt_id

having count(t1.item_id) = 3)
'''
cur.execute(sql2)
pd.DataFrame(cur.fetchall())

,0
0,22


Для > 3

In [95]:
sql2 = '''

select count(*) from

(select

t1.receipt_id,
count(t1.item_id)

from sales t1

where
t1.week = 3

group by
t1.receipt_id

having count(t1.item_id) > 3)
'''
cur.execute(sql2)
pd.DataFrame(cur.fetchall())

,0
0,173


Обязательно посчитай 